In [1]:
# run setup for notebook
%load_ext rpy2.ipython
%config InlineBackend.figure_format='retina'

In [3]:
import numpy as np
import matplotlib.pyplot as plt

import gpflow
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions